In [22]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.metrics import plot_confusion_matrix, f1_score, roc_curve

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

import pickle

In [2]:
# Import Data
df = pd.read_csv('data/baseball_no_nulls_after_dan2.csv') #2015-2020 data
df.iloc[:,1:2] = df.iloc[:,1:2].astype(np.float32)
df.iloc[:,4:] = df.iloc[:,4:].astype(np.float32)

In [4]:
# DO NOT RUN AGAIN
batter_dummies.shape[0] == df.shape[0] # confirmed shape, so now I will just concat immediately

True

In [3]:
# Create A threshold
# counts = df['batter_id'].value_counts()
# repl = counts[counts <= 20].index # 287 of them do not matter to me
# df = pd.concat([df,pd.get_dummies(df['batter_id'].replace(repl, 'uncommon'))], axis=1)
df.head(3)

,Pitcher_name,batter_id,pitch_name,strike_attempt,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage
0,Wade Davis,527038.0,4-Seam Fastball,strike,96.099998,-1.92,5.95,1.0,1.0,1.0,2.0,-0.27,1.70,-0.93,1.62,12.0,0.0,95.400002,2463.0,6.0,2.0,7.0,3.075,-139.608994,-9.724,-2.028,34.779999,-11.706,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
1,Wade Davis,527038.0,Cutter,strike,93.099998,-1.50,6.02,1.0,1.0,1.0,2.0,0.24,0.72,0.78,1.55,12.0,0.0,92.199997,2705.0,5.0,2.0,7.0,5.156,-135.442001,-7.227,5.004,28.834999,-25.323,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
2,Wade Davis,527038.0,4-Seam Fastball,strike,97.000000,-1.37,6.08,1.0,1.0,1.0,2.0,-0.50,1.41,1.32,2.47,12.0,0.0,96.400002,2362.0,4.0,2.0,7.0,8.459,-140.843002,-7.313,-5.412,31.551001,-15.189,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015


In [3]:
ignore = ['Pitcher_name','batter_id','strike_attempt']
ignore2 = ['Pitcher_name','strike_attempt']
y = df['strike_attempt']

In [6]:

X = df.loc[:,'release_speed':]
X.head()

,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage
0,96.099998,-1.92,5.95,1.0,1.0,1.0,2.0,-0.27,1.70,-0.93,1.62,12.0,0.0,95.400002,2463.0,6.0,2.0,7.0,3.075,-139.608994,-9.724,-2.028,34.779999,-11.706,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
1,93.099998,-1.50,6.02,1.0,1.0,1.0,2.0,0.24,0.72,0.78,1.55,12.0,0.0,92.199997,2705.0,5.0,2.0,7.0,5.156,-135.442001,-7.227,5.004,28.834999,-25.323,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
2,97.000000,-1.37,6.08,1.0,1.0,1.0,2.0,-0.50,1.41,1.32,2.47,12.0,0.0,96.400002,2362.0,4.0,2.0,7.0,8.459,-140.843002,-7.313,-5.412,31.551001,-15.189,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
3,93.599998,-1.25,6.06,1.0,1.0,0.0,2.0,0.32,0.80,2.92,2.23,12.0,0.0,92.800003,2724.0,3.0,2.0,7.0,10.094,-135.733994,-5.821,6.079,26.355000,-24.250,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
4,97.099998,-1.49,6.08,1.0,1.0,0.0,1.0,-0.76,1.63,0.26,2.74,12.0,0.0,96.000000,2401.0,2.0,2.0,7.0,6.519,-140.992996,-7.138,-9.148,32.250999,-11.956,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=66, stratify=y)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2824295, 42), (941432, 42), (2824295,), (941432,))

In [23]:
X2 = df[[col for col in df.columns if col not in ignore]]
X2.head()

,pitch_name,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage
0,4-Seam Fastball,96.099998,-1.92,5.95,1.0,1.0,1.0,2.0,-0.27,1.70,-0.93,1.62,12.0,0.0,95.400002,2463.0,6.0,2.0,7.0,3.075,-139.608994,-9.724,-2.028,34.779999,-11.706,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
1,Cutter,93.099998,-1.50,6.02,1.0,1.0,1.0,2.0,0.24,0.72,0.78,1.55,12.0,0.0,92.199997,2705.0,5.0,2.0,7.0,5.156,-135.442001,-7.227,5.004,28.834999,-25.323,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
2,4-Seam Fastball,97.000000,-1.37,6.08,1.0,1.0,1.0,2.0,-0.50,1.41,1.32,2.47,12.0,0.0,96.400002,2362.0,4.0,2.0,7.0,8.459,-140.843002,-7.313,-5.412,31.551001,-15.189,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
3,Cutter,93.599998,-1.25,6.06,1.0,1.0,0.0,2.0,0.32,0.80,2.92,2.23,12.0,0.0,92.800003,2724.0,3.0,2.0,7.0,10.094,-135.733994,-5.821,6.079,26.355000,-24.250,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
4,4-Seam Fastball,97.099998,-1.49,6.08,1.0,1.0,0.0,1.0,-0.76,1.63,0.26,2.74,12.0,0.0,96.000000,2401.0,2.0,2.0,7.0,6.519,-140.992996,-7.138,-9.148,32.250999,-11.956,2.0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015


In [24]:
X2 = pd.get_dummies(X2, columns=['pitch_name'])

In [25]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, random_state=66, stratify=y)

In [16]:
pitcher = df[[col for col in df.columns if "usage" in col]]
pitcher.head()

,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage
0,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
1,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
2,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
3,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015
4,0.018929,0.47782,0.005354,0.000191,0.293881,0.0,0.0,0.0,0.000765,0.19847,0.0,0.000191,0.0,0.0,0.000382,0.0,0.004015


In [8]:
# pitcher = pd.get_dummies(pitcher, columns=['Pitcher_name','pitch_name']) 
# only batter and the pitch that's being thrown at them

In [17]:
pitcher_train, pitcher_test, y3_train, y3_test = train_test_split(pitcher, y, random_state=66, stratify=y)

# Random Forest

---
The features not included are Pitcher name, batter id, and pitch name.

In [18]:
params = ({
    'n_estimators':[100,150,200],
    'max_depth':[2,3,4,5],
    'max_features':['log2','auto']
})

In [16]:
rf = RandomizedSearchCV(RandomForestClassifier(),
                       param_distributions=params,
                       n_jobs=8,
                       verbose=2,
                       random_state=66)

In [17]:
rf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 22.3min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed: 45.7min finished


RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
                   param_distributions={'max_depth': [2, 3, 4, 5],
                                        'max_features': ['log2', 'auto'],
                                        'n_estimators': [100, 125, 150]},
                   random_state=66, verbose=2)

In [18]:
rf.score(X_train, y_train)

0.653994713725018

In [19]:
rf.score(X_test, y_test)

0.6541130957944918

In [21]:
rf.best_params_

{'n_estimators': 150, 'max_features': 'auto', 'max_depth': 5}

In [23]:
preds = rf.predict(X_test)

In [24]:
accuracy_score(y_test, preds)

0.6541130957944918

In [75]:
# RF Model without pitch name dummied
# pickle.dump(rf, open('assets/rf_phys_120920.pkl','wb'))

In [30]:
# Using the same parameters but on a GridSearch
rf2 = GridSearchCV(RandomForestClassifier(random_state=66),
                       param_grid=params,
                       n_jobs=8,
                       verbose=2)

In [32]:
rf2.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 19.4min
[Parallel(n_jobs=8)]: Done 160 out of 160 | elapsed: 200.0min finished


GridSearchCV(estimator=RandomForestClassifier(random_state=66), n_jobs=8,
             param_grid={'max_depth': [2, 3, 4, 5],
                         'max_features': ['log2', 'auto'],
                         'n_estimators': [100, 125, 150, 175]},
             verbose=2)

In [45]:
print(f'Train score: {rf2.score(X_train, y_train)}')

Train score: 0.6469791576304883


In [46]:
print(f'Test score: {rf2.score(X_test, y_test)}')

Test score: 0.6467838356886105


In [47]:
rf2.best_params_

{'max_depth': 5, 'max_features': 'auto', 'n_estimators': 175}

In [49]:
preds2 = rf2.predict(X_test)

In [50]:
accuracy_score(y_test, preds2)

0.6467838356886105

These models will included the dummified pitch name at the very least.

In [52]:
X2_train.head(3)

,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage,pitch_name_2-Seam Fastball,pitch_name_4-Seam Fastball,pitch_name_Changeup,pitch_name_Curveball,pitch_name_Cutter,pitch_name_Eephus,pitch_name_Fastball,pitch_name_Forkball,pitch_name_Intentional Ball,pitch_name_Knuckle Curve,pitch_name_Knuckleball,pitch_name_Pitch Out,pitch_name_Screwball,pitch_name_Sinker,pitch_name_Slider,pitch_name_Split-Finger,pitch_name_Unknown
949279,86.000000,1.98,6.24,1.0,0.0,1.0,1.0,1.20,1.10,0.66,2.14,3.0,0.0,85.900002,1898.0,3.0,1.0,4.0,-5.637918,-125.059944,-5.664565,14.201257,24.067158,-19.9678,0.0,0.155011,0.531297,0.110244,0.04883,0.000000,0.0,0.0,0.0,0.000983,0.000000,0.0,0.000000,0.0,0.013174,0.139870,0.0,0.000590,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2434923,90.300003,-2.48,5.74,0.0,1.0,0.0,0.0,-1.38,1.21,-0.08,2.61,3.0,1.0,88.199997,2097.0,1.0,0.0,5.0,9.228000,-131.164001,-4.304000,-15.697000,27.677999,-19.7810,2.0,0.222577,0.308790,0.030992,0.00000,0.164726,0.0,0.0,0.0,0.003005,0.169421,0.0,0.000188,0.0,0.016153,0.083396,0.0,0.000751,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1117409,97.599998,-2.46,6.19,1.0,1.0,0.0,0.0,-0.50,1.70,-1.18,3.60,3.0,0.0,97.900002,2391.0,1.0,1.0,1.0,3.926000,-141.953003,-4.819000,-5.253000,33.480999,-10.4950,2.0,0.000534,0.516814,0.132166,0.00000,0.006192,0.0,0.0,0.0,0.000427,0.000000,0.0,0.000000,0.0,0.000000,0.341732,0.0,0.002135,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
rf3 = RandomizedSearchCV(RandomForestClassifier(),
                       param_distributions=params,
                       n_jobs=8,
                       verbose=2,
                       random_state=66)

In [55]:
rf3.fit(X2_train, y2_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed: 21.1min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed: 48.9min finished


RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
                   param_distributions={'max_depth': [2, 3, 4, 5],
                                        'max_features': ['log2', 'auto'],
                                        'n_estimators': [100, 150, 200]},
                   random_state=66, verbose=2)

In [56]:
print(f'Train score: {rf3.score(X2_train, y2_train)}')
print(f'Test score: {rf3.score(X2_test, y2_test)}')

Train score: 0.6392076606728405
Test score: 0.6388916034296689


The scores dropped compared to when I didn't include the pitch that was thrown.

In [58]:
preds3 = rf3.predict(X2_test)
accuracy_score(y_test, preds3)

0.6388916034296689

In [68]:
rf3.best_params_

{'n_estimators': 200, 'max_features': 'auto', 'max_depth': 5}

In [74]:
# Save the model with a 63% accuracy score
# pickle.dump(rf3, open('assets/rf_phys_pitch_name_120920.pkl','wb'))

I am going to run a model based on the batters coming up.

In [13]:
# params = ({
#     'n_estimators':[100,200],
#     'max_depth':[4,5],
#     'max_features':['auto']
# })
rf4 = RandomizedSearchCV(RandomForestClassifier(),
                       param_distributions=params,
                       n_jobs=8,
                       verbose=2,
                       random_state=66)

In [19]:
# Running only the attributes of what makes each pitcher unique
rf4.fit(pitcher_train, y3_train)

C:\Users\ncesm\anaconda3\envs\capstone\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 out of  20 | elapsed: 19.3min remaining:  4.8min
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed: 22.7min finished


RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
                   param_distributions={'max_depth': [4, 5],
                                        'max_features': ['auto'],
                                        'n_estimators': [100, 200]},
                   random_state=66, verbose=2)

In [20]:
print(f'Train score: {rf4.score(pitcher_train, y3_train)}')
print(f'Test score: {rf4.score(pitcher_test, y3_test)}')

Train score: 0.4600772936254888
Test score: 0.4600778388667477


In [21]:
rf4.best_params_

{'n_estimators': 100, 'max_features': 'auto', 'max_depth': 4}

# Logistic Regression

---
Try to use the same parameters but with LogReg

In [80]:
pipe = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(n_jobs=8))])

In [81]:
params2 = ({
    'model__penalty':['l1','l2'],
    'model__C':[.90,95,.99,1],
    'model__max_iter':[100,200,300]
})

In [83]:
lr = RandomizedSearchCV(pipe,
                       param_distributions=params2,
                       n_jobs=8,
                       verbose=1,
                       random_state=66)


In [84]:
lr.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 13.4min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed: 19.3min finished


RandomizedSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('model',
                                              LogisticRegression(n_jobs=8))]),
                   n_jobs=8,
                   param_distributions={'model__C': [0.9, 95, 0.99, 1],
                                        'model__max_iter': [100, 200, 300],
                                        'model__penalty': ['l1', 'l2']},
                   random_state=66, verbose=1)

In [85]:
# Scaled
print(f'Train score: {lr.score(X_train, y_train)}')
print(f'Test score: {lr.score(X_test, y_test)}')

Train score: 0.5109080319159295
Test score: 0.5104691576237051


In [63]:
# Unscaled
print(f'Train score: {lr.score(X_train, y_train)}')
print(f'Test score: {lr.score(X_test, y_test)}')

Train score: 0.5250828967937131
Test score: 0.5249704705172545


In [86]:
lr.best_params_

{'model__penalty': 'l2', 'model__max_iter': 100, 'model__C': 0.9}

In [87]:
lr2 = RandomizedSearchCV(pipe,
                       param_distributions=params2,
                       n_jobs=8,
                       verbose=1,
                       random_state=66)

In [88]:
lr2.fit(X2_train, y2_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 15.3min
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed: 22.2min finished


RandomizedSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('model',
                                              LogisticRegression(n_jobs=8))]),
                   n_jobs=8,
                   param_distributions={'model__C': [0.9, 95, 0.99, 1],
                                        'model__max_iter': [100, 200, 300],
                                        'model__penalty': ['l1', 'l2']},
                   random_state=66, verbose=1)

In [89]:
# scaled
print(f'Train score: {lr2.score(X2_train, y2_train)}')
print(f'Test score: {lr2.score(X2_test, y2_test)}')

Train score: 0.5118990756985372
Test score: 0.511550489042225


In [72]:
# unscale
print(f'Train score: {lr2.score(X2_train, y2_train)}')
print(f'Test score: {lr2.score(X2_test, y2_test)}')

Train score: 0.5212791156731149
Test score: 0.5210976469888425


In [90]:
lr2.best_params_

{'model__penalty': 'l2', 'model__max_iter': 100, 'model__C': 0.9}

In [96]:
# 51% accuracy
# pickle.dump(lr, open('assets/lr_phys_scaled_120920.pkl','wb'))
# pickle.dump(lr2, open('assets/lr_phys_pitchname_scaled_120920.pkl','wb'))

# XGBoosting

In [26]:
X2_train.head(3)

,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,2s_usage,4s_usage,changeup_usage,curveball_usage,cutter_usage,eephus_usage,fastball_usage,forkball_usage,ball_usage,knucklecurve_usage,knuckleball_usage,pitchout_usage,screwball_usage,sinker_usage,slider_usage,split_usage,unknown_usage,pitch_name_2-Seam Fastball,pitch_name_4-Seam Fastball,pitch_name_Changeup,pitch_name_Curveball,pitch_name_Cutter,pitch_name_Eephus,pitch_name_Fastball,pitch_name_Forkball,pitch_name_Intentional Ball,pitch_name_Knuckle Curve,pitch_name_Knuckleball,pitch_name_Pitch Out,pitch_name_Screwball,pitch_name_Sinker,pitch_name_Slider,pitch_name_Split-Finger,pitch_name_Unknown
949279,86.000000,1.98,6.24,1.0,0.0,1.0,1.0,1.20,1.10,0.66,2.14,3.0,0.0,85.900002,1898.0,3.0,1.0,4.0,-5.637918,-125.059944,-5.664565,14.201257,24.067158,-19.9678,0.0,0.155011,0.531297,0.110244,0.04883,0.000000,0.0,0.0,0.0,0.000983,0.000000,0.0,0.000000,0.0,0.013174,0.139870,0.0,0.000590,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2434923,90.300003,-2.48,5.74,0.0,1.0,0.0,0.0,-1.38,1.21,-0.08,2.61,3.0,1.0,88.199997,2097.0,1.0,0.0,5.0,9.228000,-131.164001,-4.304000,-15.697000,27.677999,-19.7810,2.0,0.222577,0.308790,0.030992,0.00000,0.164726,0.0,0.0,0.0,0.003005,0.169421,0.0,0.000188,0.0,0.016153,0.083396,0.0,0.000751,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1117409,97.599998,-2.46,6.19,1.0,1.0,0.0,0.0,-0.50,1.70,-1.18,3.60,3.0,0.0,97.900002,2391.0,1.0,1.0,1.0,3.926000,-141.953003,-4.819000,-5.253000,33.480999,-10.4950,2.0,0.000534,0.516814,0.132166,0.00000,0.006192,0.0,0.0,0.0,0.000427,0.000000,0.0,0.000000,0.0,0.000000,0.341732,0.0,0.002135,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
params = ({
    'n_estimators':[100,150,200],
    'max_depth':[3,4,5,6],
    'learning_rate':[.3,.5,.7],
    'max_delta_step':[0,1,2]
})

In [ ]:
xgb = GridSearchCV(XGBClassifier(random_state=66),
                       param_grid=params,
                       n_jobs=8,
                       verbose=2)
xgb.fit(X2_train, y2_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
